In [ ]:
from constants.abs_paths import AbsDirPath
import pandas as pd

In [ ]:
file_dir = AbsDirPath.KEYWORDS_MATCHING

def load_all_files(dir):
    dfs = []
    for file_path in dir.glob("*.parquet"):
        try:
            file = pd.read_parquet(file_path)
            # file['fname'] = file_path
            dfs.append(file)
            print(f"Loaded {file_path}")
        except:
            print(f"Error while loading {file_path}")

    df = pd.concat(dfs)
    return df


df = load_all_files(file_dir)

In [ ]:
from IPython.display import display

def display_and_save(df, path):
    df.to_excel(path, merge_cells=False)
    display(df)

def display_and_save_csv(df, path):
    df.to_csv(path)
    display(df)

In [ ]:
df.head()

In [ ]:
df.shape

## Raw data overview


In [ ]:
display_and_save(df.pivot_table(index=["source"], columns=["qa"], values="keyword", aggfunc="count", fill_value=0, margins=True).sort_values(by="All", ascending=False), AbsDirPath.RES_KEYWORDS_MATCHING / "pivot_source_to_qa.xlsx")

In [ ]:
display_and_save(df.pivot_table(index=["source", "repo_id"], columns=["qa"], values="keyword", aggfunc="count", fill_value=0, margins=True), AbsDirPath.RES_KEYWORDS_MATCHING / "pivot_source&repo_to_qa.xlsx")

## Duplicates


In [ ]:
df.groupby(["sentence", "source", "keyword"])['matched_word'].count().sort_values(ascending=False)

In [ ]:
df["sent_kw_n"] = df.groupby(["sentence", "keyword"]).transform('size')

In [ ]:
display_and_save_csv(df[df.sent_kw_n > 50].groupby(["sentence", "keyword"]).first().sort_values(by="sent_kw_n", ascending=False).reset_index(), AbsDirPath.RES_KEYWORDS_MATCHING / "top_duplicate_sentence&kw.csv")

In [ ]:
df.groupby(["sentence", "keyword"]).size().rename("size")[lambda x: x > 100].sort_values(ascending=False).first()

In [ ]:
df[df.groupby(["sentence", "keyword"])['matched_word'].transform('count') > 100].groupby(["sentence", "keyword"]).first()

In [ ]:
df[df.groupby(["sentence", "keyword"]).transform('size') > 100]

In [ ]:
df.groupby(["sentence", "keyword"]).first()

In [ ]:
df.groupby(["sentence", "source", "keyword", "url"])['id'].count().sort_values(ascending=False)

In [ ]:
df['sentence_counts'] = df.groupby(["sentence", "source", "keyword"])['id'].transform('count')

## Optimizations

In [ ]:
import hashlib

quick_hash = lambda x: hashlib.md5(x.encode('utf-8')).hexdigest()

df['text_hash'] = df['sentence'].apply(quick_hash)

In [ ]:
df[["repo_id", "source", "qa", "text_hash", "keyword", "matched_word"]].agg(";".join, axis=1)

In [ ]:
df['id'] = df.repo_id + "." + df.source + "." + df.qa + "." +  df.text_hash + "." + df['keyword'] + "." +df['matched_word']
df['id_lower_mword'] = df.repo_id + "." + df.source + "." + df.qa + "." +  df.text_hash + "." + df['keyword'] + "." +df['matched_word'].str.lower()

In [ ]:
df.groupby('id').size().sort_values(ascending=False)

In [ ]:
df.groupby('id_lower_mword').size().sort_values(ascending=False)

In [ ]:
df.groupby('id_lower_mword')

In [ ]:
df.id_lower_mword.nunique()

In [ ]:
1602564 / 1994850

In [ ]:
df['matched_word_lower'] = df['matched_word'].str.lower()

In [ ]:
df.groupby('id_lower_mword').matched_word.agg(['unique','nunique']).sort_values(by="nunique", ascending=False)

In [ ]:
df.groupby('id_lower_mword').matched_word.agg(['unique','nunique', 'count']).sort_values(by=['count',"nunique"], ascending=[False, False])

In [ ]:
df['similar_count'] = df.groupby(["real_id"])['id'].transform('count')

In [ ]:
df2 = df.copy()

In [ ]:
df2['similar_count'] = df2.groupby(["id_lower_mword"]).transform('size')

In [ ]:
df2['all_matched'] = df2['id_lower_mword'].map(df2.groupby("id_lower_mword").matched_word.agg(lambda x: list(x.unique())))

In [ ]:
df2['n_all_matched'] = df2.groupby("id_lower_mword").matched_word.transform('nunique')

In [ ]:
df2.groupby("sentence").agg(similar_count=("similar_count", "sum"), total_sentences=("keyword", "count")).sort_values(by=["total_sentences", "similar_count"], ascending=[False] * 2)

In [ ]:
df2["n_same_sentences"] = df2.groupby(["sentence"]).transform("size")

In [ ]:
# TODO: do this before hashing sentences and assigning ids
df2.sentence = df2.sentence.str.replace(r"((\t|\r) ?)+", " ", regex=True)

In [ ]:
df2.sentence = df2.sentence.str.replace(r"[ \"'#]{2,}", " ", regex=True)

In [ ]:
df2.groupby(["sentence"]).first().sort_values(["sentence", "n_same_sentences", "similar_count"], ascending=[True, False, False])

In [ ]:
df2.sort_values(["sentence", "n_same_sentences", "similar_count"], ascending=[True, False, False])

In [ ]:
df2 = df2.groupby(['id_lower_mword']).first()

In [ ]:
df2.head()

In [ ]:
df[df.sentence.str.startswith("@phsft-bot build")]

In [ ]:
df[df.sentence.str.startswith("Build failed")]

In [ ]:
# Potentially can be eliminated
df[df.sentence.str.startswith("Starting build")]

In [ ]:
df.matched_word = df.matched_word.str.lower()

In [ ]:
df[df.qa == "security"].pivot_table(index=["keyword", "matched_word"], columns=["repo_id"],values=["similar_count"], aggfunc="sum", fill_value=0, margins=True).sort_values(by=("similar_count", "All"),ascending=False)

In [ ]:
df.pivot_table(index=["keyword", "matched_word"], values=["similar_count"], columns=["qa"], aggfunc="sum", fill_value=0, margins=True).sort_values(by=("similar_count", "All"),ascending=False)

In [ ]:
df.groupby("keyword").matched_word.nunique().sort_values(ascending=False)

In [ ]:
df.groupby(["keyword", "matched_word"]).similar_count.sum()

In [ ]:
df.head()

In [ ]:
# check how reduced dataset gets
df.reset_index().groupby(["sentence"]).agg({'similar_count': 'sum', 'real_id': 'count'}).nlargest(1000, 'similar_count')

In [ ]:
df.groupby("keyword").matched_word.agg(['unique','nunique']).sort_values(by="nunique", ascending=False)

In [ ]:
df.groupby(["qa", "keyword"]).matched_word.agg(['unique','nunique']).sort_values(by="nunique", ascending=False)

In [ ]:
df.groupby(["keyword", "qa"]).matched_word.agg(['unique','nunique'])

In [ ]:
df.groupby(["keyword", "qa", "matched_word"])['sentence'].apply(lambda x: x.head(3).to_list())

In [ ]:
df.groupby(["keyword", "qa"]).agg(
    matched_words=("matched_word", 'unique'),
    nunique=("matched_word", 'nunique'),
    total_matched=("similar_count", "sum")
)

In [ ]:
df.groupby(["keyword", "qa"]).agg(
    matched_words=("matched_word", 'unique'),
    nunique=("matched_word", 'nunique'),
    total_matched=("similar_count", "sum")
).reset_index().to_csv(AbsDirPath.KEYWORD_ANALYSIS / "all_matched_words_by_keywords_&_QAs.csv", index=False, header=True)

In [ ]:
import itertools

sources = pd.Series(df.source.unique()).str.split(".").str[0].tolist()
repos = df.repo_id.unique()
output_dir = AbsDirPath.O_KEYWORDS_MATCHING
output_dir.mkdir(exist_ok=True)

for source, repo_id in itertools.product(sources, repos):
    output_file = output_dir / f"{repo_id}.{source}.parquet"
    print(output_file)
    df[(df.source == source) & (df.repo_id == repo_id)].drop(columns=["id", "fname", "sentence_counts"]).to_parquet(output_file, engine='pyarrow', compression='snappy', index=False)
    print(f"Saved {output_file}")

In [ ]:
df